In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np 
import pandas as pd 
import math
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler, OrdinalEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, GridSearchCV
from sklearn.svm import SVR
import scipy.stats
import os
from sklearn.impute import SimpleImputer
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn import metrics
!pip install catboost
import catboost as cb
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.impute import SimpleImputer
from sklearn.neural_network import MLPClassifier
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import accuracy_score
!pip install unique_names_generator

In [ ]:
train_df = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
test_df = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')

In [ ]:
dataset = pd.concat([train_df, test_df], ignore_index=True)

In [ ]:
train_df.info()

In [ ]:
df = dataset.copy()
df[['Deck','Cab_Number','Side']] = df['Cabin'].str.split("/", expand=True)
df[["Group", "Group_num"]] = df['PassengerId'].str.split("_", expand=True)
df[['Name', 'Family']] = df['Name'].str.split(" ",expand=True)
df.drop(["Cabin", "Name"], axis=1, inplace=True) 
df[["Group", "Group_num"]] = df[["Group", "Group_num"]].astype(int)
df.drop(['Cab_Number'],  axis=1, inplace=True)

In [ ]:
df.info()

# Missings

In [ ]:
df[(df['Age']<13)]['RoomService'].unique()

In [ ]:
df[(df['Age']<13)]['FoodCourt'].unique()

In [ ]:
df[(df['Age']<13)]['ShoppingMall'].unique()

In [ ]:
df[(df['Age']<13)]['Spa'].unique()

In [ ]:
df[(df['Age']<13)]['VRDeck'].unique()

In [ ]:
service = ["RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck"]
values = dict(zip(service, [0.0]*5))
values['VIP'] = False
df.loc[df.Age < 13] = df.loc[df.Age < 13].fillna(value=values)

In [ ]:
from unique_names_generator import get_random_name

def family_fill(item):
    fam_df = df.loc[df.Group == item]
    if len(fam_df) > 1:
        return fam_df.Family.describe().top
    else:
        random  = get_random_name().split()[1]
        return random

df.Family.loc[df.Family.isnull()] = df.loc[df.Family.isnull()]['Group'].apply(family_fill)

In [ ]:
df

In [ ]:
values = dict(zip(service, [0.0]*5))
df.loc[df.CryoSleep == True] = df.loc[df.CryoSleep == True].fillna(values)

In [ ]:
def cryosleep_val(item):
    if item == 0.0:
        return True
    else:
        return False

df.CryoSleep.loc[df.CryoSleep.isnull()] = df.loc[df.CryoSleep.isnull()][service].sum(axis=1).apply(cryosleep_val)

In [ ]:
df.loc[(df.VIP.isnull()) & (df.CryoSleep == True)] = df.loc[(df.VIP.isnull()) & (df.CryoSleep == True)].fillna({'VIP':False})

In [ ]:
def home(item):
    data = df.loc[df.Family == item]
    if len(data.loc[data.HomePlanet.notnull()]) > 1:
        return data.HomePlanet.describe().top
    else:
        return df.HomePlanet.describe().top

df.HomePlanet.loc[df.HomePlanet.isnull()] = df.loc[df.HomePlanet.isnull()]['Family'].apply(home)

In [ ]:
def dest(item):
    data = df.loc[df.Family == item]
    if len(data) > 1:
        return data.Destination.describe().top
    else:
        return df.Destination.describe().top

df.Destination.loc[df.Destination.isnull()] = df.loc[df.Destination.isnull()]['Family'].apply(dest)

In [ ]:
no_vip = df.loc[(df.VIP ==False) & (df.CryoSleep == False)]
no_vip[service].sum(axis=1).mean()

In [ ]:
def VIP_mean(item):
    mean = 2149.8388002552647
    if item > mean:
        return True
    else:
        return False

df.VIP.loc[(df.VIP.isnull()) & (df.CryoSleep == False)] = df.loc[(df.VIP.isnull()) & (df.CryoSleep == False)][service].sum(axis=1).apply(VIP_mean)

In [ ]:
def deck_fill(item):
    group_df = df.loc[df.Group == item]
    if len(group_df.loc[group_df.Deck.notnull()]) > 1:
        return group_df.Deck.describe().top
    else:
        return df.Deck.describe().top

def side_fill(item):
    group_df = df.loc[df.Group == item]
    if len(group_df.loc[group_df.Side.notnull()]) > 1:
        return group_df.Side.describe().top
    else:
        return df.Side.describe().top


df['Deck'].loc[(df.Deck.isnull())] = df.loc[(df.Deck.isnull())]['Group'].apply(deck_fill)
df['Side'].loc[(df.Side.isnull())]= df.loc[(df.Side.isnull())]['Group'].apply(side_fill)

In [ ]:
def age_fill(item):
    group = df.loc[df.Group == item]
    if len(group.loc[group.Age.notnull()]) > 1:
        return group.Age.describe()['mean'].round()
    else:
        return df.Age.describe()['mean'].round()

df.Age.loc[(df.Age.isnull())& (df.Group.notnull())] = df.loc[(df.Age.isnull())& (df.Group.notnull())]['Group'].apply(age_fill)

In [ ]:
def roomservice_fill(item):
    group = df.loc[df.Group == item]
    if len(group.loc[group.RoomService.notnull()]) > 1:
        return int(group.RoomService.mean())
    else:
        return 0.0 

df.RoomService.loc[(df.RoomService.isnull())] = df.loc[(df.RoomService.isnull())]['Group'].apply(roomservice_fill)

def food_fill(item):
    group = df.loc[df.Group == item]
    if len(group.loc[group.FoodCourt.notnull()]) > 1:
        return int(group.FoodCourt.mean())
    else:
        return 0.0

df.FoodCourt.loc[(df.FoodCourt.isnull())] = df.loc[(df.FoodCourt.isnull())]['Group'].apply(food_fill)

def shopping_fill(item):
    group = df.loc[df.Group == item]
    if len(group.loc[group.ShoppingMall.notnull()]) > 1:
        return int(group.ShoppingMall.mean())
    else:
        return 0.0

df.ShoppingMall.loc[(df.ShoppingMall.isnull())] = df.loc[(df.ShoppingMall.isnull())]['Group'].apply(shopping_fill)

def spa_fill(item):
    group = df.loc[df.Group == item]
    if len(group.loc[group.Spa.notnull()]) > 1:
        return int(group.Spa.mean())
    else:
        return 0.0

df.Spa.loc[(df.Spa.isnull())] = df.loc[(df.Spa.isnull())]['Group'].apply(spa_fill)

def vrdeck_fill(item):
    group = df.loc[df.Group == item]
    if len(group.loc[group.VRDeck.notnull()]) > 1:
        return int(group.VRDeck.mean())
    else:
        return 0.0 

df.VRDeck.loc[(df.VRDeck.isnull())] = df.loc[(df.VRDeck.isnull())]['Group'].apply(vrdeck_fill)

In [ ]:
df.info()

In [ ]:
train = df.iloc[:8693,:]
test = df.iloc[8693:,:]

In [ ]:
train

# Descriptive statistics

In [ ]:
sns.countplot(train.HomePlanet, order = train['HomePlanet'].value_counts().index)
plt.title('Number of passengers per Home Planet');

In [ ]:
sns.countplot(train.HomePlanet, hue=train.Transported, order = train['HomePlanet'].value_counts().index)
plt.title('Number of transported passengers by Home Planet');

In [ ]:
sns.countplot(train.Destination, order = train['Destination'].value_counts().index)
plt.title('Number of passengers per Destination');

In [ ]:
sns.countplot(hue='Transported', x='Destination', data=train, order = train['Destination'].value_counts().index)
plt.title('Number of transported passengers by destination');

In [ ]:
sns.countplot(train.CryoSleep)
plt.title('Number of Passengers in CryoSleep');

In [ ]:
sns.countplot(x='CryoSleep', hue='Transported', data=train)
plt.title('Number of transported passengers by CryoSleep');

In [ ]:
sns.countplot(train.VIP)
plt.title('Number of VIP passengers');

In [ ]:
sns.countplot(hue='Transported', x='VIP', data=train)
plt.title('Number of transported passengers by VIP status');

In [ ]:
sns.countplot(train.Deck, order = train['Deck'].value_counts().index)
plt.title('Number of Passengers by Deck');

In [ ]:
sns.countplot(hue='Transported', x='Deck', data=train, order = train['Deck'].value_counts().index)
plt.title('Number of transported passengers by deck');

In [ ]:
sns.countplot(hue='VIP', x='Deck', data=train, order = train['Deck'].value_counts().index)
plt.title('VIP passengers by deck');

In [ ]:
train0 = train.copy()
train0['services']=train["RoomService"] + train["FoodCourt"] + train["ShoppingMall"] + train["Spa"] + train["VRDeck"]
train00=train0.groupby('Deck').agg('sum').sort_values(['services'], ascending=False)   
train00=train00.iloc[:,8:]
fig, ax1 = plt.subplots(ncols=1, nrows=1, figsize=(10, 10))
train00.plot.bar(ax=ax1)
ax1.set_title("Total amount of services by deck", size=20)
ax1.set_xlabel("Deck", size=15)
ax1.set_ylabel("Amount of services", size=15)
for tick in ax1.xaxis.get_major_ticks():
    tick.label.set_fontsize(15)
for tick in ax1.yaxis.get_major_ticks():
    tick.label.set_fontsize(15)
plt.xticks(rotation=0)
plt.show()

In [ ]:
sns.countplot(train.Side)
plt.title('Number of Passengers by side');

In [ ]:
sns.countplot(x='Side', hue='Transported', data=train)
plt.title('Number of transported passengers by side');

# Data preparation

In [ ]:
train['CryoSleep'] = train['CryoSleep'].replace("False", value=0)
train['CryoSleep'] = train['CryoSleep'].replace("True", value=1)
train['CryoSleep'] = train['CryoSleep'].astype('int')

test['CryoSleep'] = test['CryoSleep'].replace("False", value=0)
test['CryoSleep'] = test['CryoSleep'].replace("True", value=1)
test['CryoSleep'] = test['CryoSleep'].astype('int')

In [ ]:
train['VIP'] = train['VIP'].replace("False", value=0)
train['VIP'] = train['VIP'].replace("True", value=1)
train['VIP'] = train['VIP'].astype('int')

test['VIP'] = test['VIP'].replace("False", value=0)
test['VIP'] = test['VIP'].replace("True", value=1)
test['VIP'] = test['VIP'].astype('int')

In [ ]:
train['Transported'] = train['Transported'].replace(False, value=0)
train['Transported'] = train['Transported'].replace(True, value=1)

In [ ]:
label_cols = ['HomePlanet','Destination', 'Deck',	'Side']
def label_encoder(df,df1, columns):
    for col in columns:
        df[col] = df[col].astype(str)
        df1[col] = df1[col].astype(str)
        df[col] = LabelEncoder().fit_transform(df[col])
        df1[col] = LabelEncoder().fit_transform(df1[col])
    return df, df1

train_1, test_1 = label_encoder(train, test, label_cols)

In [ ]:
train_1['Group_num'].value_counts()

In [ ]:
test_1['Group_num'].value_counts()

In [ ]:
train_1.drop(['Group', 'Family'],  axis=1, inplace=True)
test_1.drop(['Group', 'Family'],  axis=1, inplace=True)

train_1.drop(['Group_num'],  axis=1, inplace=True)
test_1.drop(['Group_num'],  axis=1, inplace=True)

In [ ]:
train_1['Age'].describe()

In [ ]:
def age_group(data):
    if data == 0:
        return 0 #special categories for values equal to 0
    elif (data > 0) & (data <= 19):
        return 0
    elif (data > 19) & (data <= 27):
        return 1
    elif (data > 27) & (data <= 37):
        return 2
    elif (data > 37):
        return 3
    
    
train_1['Age_Group'] = train_1['Age'].apply(age_group)
train_1.drop(['Age'],  axis=1, inplace=True)

test_1['Age_Group'] = test_1['Age'].apply(age_group)
test_1.drop(['Age'],  axis=1, inplace=True)

In [ ]:
train_1.drop(['PassengerId'],  axis=1, inplace=True)
test_1.drop(['PassengerId', 'Transported'],  axis=1, inplace=True)

In [ ]:
train_1

In [ ]:
train_1.corr().style.background_gradient(cmap='coolwarm')

In [ ]:
sns.set(rc={'figure.figsize': (12, 12)})
sns.heatmap(train_1.corr(), cmap="RdBu", annot= True)
plt.xticks(rotation=45)
plt.show()

In [ ]:
train_1.hist(figsize=(20,20))

In [ ]:
sns.displot(train_1['Transported'])

# Models

## Random forest

In [ ]:
target = train_1['Transported']
features = train_1.drop('Transported', axis = 1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=50)

In [ ]:
forest = RandomForestClassifier()
forest.fit(X_train, y_train)

In [ ]:
y_pred_fg = forest.predict(X_test)
print(f1_score(y_test, y_pred_fg)) 
print(accuracy_score(y_test, y_pred_fg))

## XGBClassifier

In [ ]:
xgb = xgb.XGBClassifier()
xgb.fit(X_train, y_train)

In [ ]:
y_pred_xg = xgb.predict(X_test)
print(f1_score(y_test, y_pred_xg)) 
print(accuracy_score(y_test, y_pred_xg))

##  LGBMClassifier

In [ ]:
lgbm = lgb.LGBMClassifier()
lgbm.fit(X_train, y_train)

In [ ]:
y_pred_lg = lgbm.predict(X_test)
print(f1_score(y_test, y_pred_lg)) 
print(accuracy_score(y_test, y_pred_lg))

## Catboost

In [ ]:
X_train

In [ ]:
catboost = cb.CatBoostClassifier(objective="MultiClass",  
                                 logging_level="Silent", 
                                 bootstrap_type="Bernoulli",
                                 cat_features=[0, 1, 2, 3, 9, 10, 11],  
                                 one_hot_max_size=4)   
catboost.fit(X_train, y_train)

In [ ]:
y_pred_cat = catboost.predict(X_test)
print(f1_score(y_test, y_pred_cat)) 
print(accuracy_score(y_test, y_pred_cat))

In [ ]:
y_predict = catboost.predict(test_1)

In [ ]:
y_predict = y_predict.astype('bool')
y_predict=y_predict[:, 0]

In [ ]:
Transported = pd.Series(y_predict)
Transported

In [ ]:
PassengerId = df.iloc[8693:,:1]
PassengerId=PassengerId.reset_index()
PassengerId.drop(['index'],  axis=1, inplace=True)
PassengerId

In [ ]:
submission = pd.concat([PassengerId, Transported], axis=1)
submission.columns = ['PassengerId', 'Transported']
submission

In [ ]:
submission.to_csv('submission.csv', index=False)